In [776]:
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import linprog
import pandas as pd
import numpy as np
import os
import json
from math import e

In [777]:
data_dir = '../data'
pd.set_option('max_columns', 70)


In [778]:
bounds_df = pd.DataFrame(index=ingredients_df.index, data={'lower': 0, 'upper': None})
bounds_df.loc['Guar gum'] = [0.9 * .01, 0.9 * .01]
bounds_df.loc['Xanthan Gum'] = [0.9 * .01, 0.9 * .01]
bounds_df.loc['Alpha-galactosidase enzyme (Beano)'] = [1.0, 1.0]
bounds_df.loc['Multivitamin'] = [1.0, 1.0]
bounds_df.loc['Corn flour, nixtamalized'] = [0, 1.0]
bounds_df.loc['Whey protein'] = [0,0.15]
bounds = list(zip(bounds_df.lower.values, bounds_df.upper.values))

In [782]:
# Download our nutrition profile data from Google Sheets
!(cd ..; make sync_data_from_google)

nutrition_profile_df = pd.read_csv(f'{data_dir}/raw/nutrition_profile.csv', index_col=0, verbose=True).transpose()
ingredients_df = pd.read_csv(f'{data_dir}/raw/ingredients.csv', index_col=0, verbose=True).transpose()

for col in ingredients_df.columns:
    ingredients_df[col] = ingredients_df[col].astype(float)

curl 'https://docs.google.com/spreadsheets/d/1UOx5ZkNMjcwybx9TgFS_EaQnjbokqDx6K9NW0mRDGlg/export?format=csv&id=1UOx5ZkNMjcwybx9TgFS_EaQnjbokqDx6K9NW0mRDGlg&gid=624419712' > data/raw/nutrition_profile.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2975    0  2975    0     0   7608      0 --:--:-- --:--:-- --:--:--  7608
curl 'https://docs.google.com/spreadsheets/d/1UOx5ZkNMjcwybx9TgFS_EaQnjbokqDx6K9NW0mRDGlg/export?format=csv&id=1UOx5ZkNMjcwybx9TgFS_EaQnjbokqDx6K9NW0mRDGlg&gid=1812860789' > data/raw/ingredients.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4875    0  4875    0     0   8705      0 --:--:-- --:--:-- --:--:--  8689
Tokenization took: 0.03 ms
Type conversion took: 0.91 ms
Parser memory cleanup took: 0.01 ms
Tokenization took: 0.04 ms
Typ

In [783]:
# Using the linprog optimizer


# Define the cost function. This can be actual cost of ingredients or a measure of the non-objectionability of ingredients to minimize
c = np.ones(ingredients_df.shape[0])

# Define the inequality constraint matrix. This is the ingredients_df data in two parts, the first
# for constraining nutrients to be greater than the RDI, the second to constrain to be less than the UL

A = -ingredients_df.transpose()[nutrition_profile_df.loc['target_scale'].ne(0)]
A = A.append(-A).values


# The upper bound inequality constraint vector. This is the nutrient UL (upper limit)

b = -nutrition_profile_df.loc['RDI'][nutrition_profile_df.loc['target_scale'].ne(0)]
b = b.append(nutrition_profile_df.loc['UL'][nutrition_profile_df.loc['target_scale'].ne(0)]).values


solution = linprog(c, A_ub=A, b_ub=b, bounds=bounds)
solution


     con: array([], dtype=float64)
     fun: 9.317958136459149
 message: 'Optimization terminated successfully.'
     nit: 25
   slack: array([1.00000000e+02, 5.47674972e-09, 7.07421098e+00, 1.20518982e+01,
       1.10650211e+01, 7.67148123e-10, 8.00575490e+00, 1.63404893e+01,
       7.94944580e+00, 1.17370655e+02, 1.90410385e+00, 1.90000000e+00,
       7.44989563e-08, 2.11001295e+01, 1.38580401e+00, 6.19141360e-08,
       2.47320130e+00, 2.84332912e-01, 1.38000000e+01, 4.83977565e+01,
       4.38012225e+00, 0.00000000e+00, 3.72016757e-07, 5.67250691e-11,
       4.20431143e+00, 0.00000000e+00, 3.05628375e+00, 2.09179509e+00,
       2.82984353e+01, 6.52921847e+02, 8.70000000e+00, 0.00000000e+00,
       1.30328463e+00, 2.11718998e-09, 1.43407211e+02, 1.11371112e-09,
       5.99999999e-01, 6.92192982e+00, 3.49268800e+02, 1.16531926e+03,
       3.79732478e+03, 7.34192129e+03, 1.99675954e+03, 7.11486086e+01,
       3.69128389e+03, 3.83519617e+03, 1.16496722e-07, 4.51996892e+03,
       1.801

In [784]:
# Scale the ingredient nutrient amounts for the given quantity of each ingredient given by the optimizer
solution_df = ingredients_df.mul(solution.x, axis=0)
solution_df.insert(0, 'Quantity (g)', solution.x * 100)

# Add a row showing the sum of the scaled amount of each nutrient
total = solution_df.sum()
total.name = 'Total'
solution_df = solution_df.append(total)
solution_df

,Quantity (g),Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
Guar gum,9.000000e-01,2.988000,0.695700,0.041400,0.004500,0.000540,0.001980,0.000630,0.000009,0.000603,0.695700,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.002646,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.001125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Vitamin K,3.125361e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,156.268047,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Oat flour,4.102223e+01,164.088919,26.664449,7.178890,3.076667,0.656356,1.189645,1.353734,0.059482,1.306558,4.102223,1.640889,2.461334,0.0,0.000000,0.051278,0.0,0.00000,0.000000,0.410222,1.312711,0.287156,0.051278,0.615333,13.127114,0.082044,0.0,1.226565e+01,0.026664,0.000000,0.0,0.179267,0.000000,1.640889,59.072011,1.640889,0.0,0.185420,0.152192,13.947558,0.007794,0.000000,1.312711,167.370697,166.140030,284.694274,526.725430,287.565831,127.989357,367.148956,235.877821,95.992018,384.378293,231.365376,483.652089,357.713843,589.079219,1525.206502,347.048064,378.635181,305.205389,0.000000
Choline Bitartrate,1.209779e-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,6.653784e-08,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Wheat bran, crude",3.745172e+01,80.895720,24.160106,5.823743,1.591698,0.235946,0.238567,0.828432,0.062544,0.763641,16.029337,0.000000,16.029337,0.0,3.370655,0.487996,0.0,0.00000,0.000000,0.558031,0.711583,0.195873,0.216096,5.085195,29.586860,0.816822,0.0,2.786408e+01,0.027340,0.000000,0.0,0.373768,0.000000,3.958647,228.830022,1.872586,0.0,0.379386,0.442679,29.062536,0.000749,0.226583,2.722740,138.945889,161.042405,182.015369,347.551980,224.710332,87.637030,222.837746,187.258610,105.613856,271.899502,163.289508,407.100218,286.505674,423.204459,1076.362491,336.316464,330.324188,256.169779,0.000000
Lecithin granules,4.907074e+00,34.349519,0.490707,0.000000,2.453537,0.736061,0.000000,1.717476,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.00000

In [626]:
nutrition_profile_df

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
RDI,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,14.5,20.4,64.6,5.1,51.0,0.0,5100.0,2.2,4.1,153.0,1020.0,34.0,204.0,2.0,2.2,27.2,680.0,8.5,51.0,935.0,1.7,3.9,35.0,0.9,150.0,8.0,420.0,2.3,45.0,0.7,4.7,55.0,1.5,0.8,11.0,816.0,780.0,650.0,910.0,2176.0,884.0,910.0,455.0,812.0,650.0,875.0,400.0,200.0,500.0,500.0,1000.0,500.0,500.0,1.0
UL,2200.0,300.0,300.0,70.0,NaN,NaN,NaN,100.0,100.0,400.0,300.0,300.0,NaN,10000.0,100.0,240.0,2000.0,4000.0,1000.0,1200.0,50.0,50.0,41.0,1000.0,500.0,500.0,3500.0,2.5,500.0,350.0,10.0,1100.0,45.0,10000.0,11.0,2000.0,4.0,50.0,400.0,2.3,2.0,40.0,1000000.0,1000000.0,1000000.0,1000000.0,300000.0,80000.0,1000000.0,100000.0,400000.0,1000000.0,1000000.0,2000.0,1000000.0,2000.0,3000.0,10000.0,1000000.0,1000000.0,1000000.0
target_fac,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.7,1.7,1.7,1.7,1.7,1.0,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
target_scale,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
RDI Base,2000.0,279.0,104.0,57.0,0.0,0.0,0.0,8.5,12.0,38.0,3.0,30.0,0.0,3000.0,1.3,2.4,90.0,600.0,20.0,120.0,1.2,1.3,16.0,400.0,5.0,30.0,550.0,1.0,2.3,35.0,0.9,150.0,8.0,420.0,2.3,45.0,0.7,4.7,55.0,1.5,0.8,11.0,816.0,780.0,650.0,910.0,2176.0,884.0,910.0,455.0,812.0,650.0,875.0,400.0,200.0,500.0,500.0,1000.0,500.0,500.0,1.0
UL Base,2200.0,300.0,300.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000.0,100.0,NaN,2000.0,4000.0,1000.0,NaN,NaN,NaN,41.0,1000.0,NaN,NaN,3500.0,2.5,NaN,NaN,10.0,1100.0,45.0,NaN,11.0,2000.0,4.0,NaN,400.0,2.3,2.0,40.0,1000000.0,NaN,NaN,NaN,300000.0,80000.0,NaN,100000.0,400000.0,NaN,NaN,2000.0,NaN,2000.0,3000.0,10000.0,NaN,NaN,NaN
target_fac_try,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.7,1.7,1.7,1.7,1.7,1.0,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.0,1.0,1.7,1.7,1.7,1.0,1.7,1.7,1.7,1.7,1.0,1.7,1.7,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4,3.4


In [463]:
ingredients_df

,Calories (kcal),Carbohydrates (g),Protein (g),Total Fat (g),Saturated Fat (g),Monounsaturated Fat (g),Polyunsaturated Fat (g),Omega-3 Fatty Acids (g),Omega-6 Fatty Acids (g),Total Fiber (g),Soluble Fiber (g),Insoluble Fiber (g),Cholesterol (mg),Vitamin A (IU),Vitamin B6 (mg),Vitamin B12 (ug),Vitamin C (mg),Vitamin D (IU),Vitamin E (IU),Vitamin K (ug),Thiamin (mg),Riboflavin (mg),Niacin (mg),Folate (ug),Pantothenic Acid (mg),Biotin (ug),Choline (mg),Calcium (g),Chloride (g),Chromium (ug),Copper (mg),Iodine (ug),Iron (mg),Magnesium (mg),Manganese (mg),Molybdenum (ug),Phosphorus (g),Potassium (g),Selenium (ug),Sodium (g),Sulfur (g),Zinc (mg),Cystine (mg),Histidine (mg),Isoleucine (mg),Leucine (mg),Lysine (mg),Methionine (mg),Phenylalanine (mg),Threonine (mg),Tryptophan (mg),Valine (mg),Tyrosine (mg),Arginine (mg),Alanine (mg),Aspartic acid (mg),Glutamic acid (mg),Glycine (mg),Proline (mg),Serine (mg),Hydroxyproline (mg)
Guar gum,332.0,77.30,4.60,0.50,0.060,0.220,0.070,0.001,0.067,77.3,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.294,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.125,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Vitamin K,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,500.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Oat flour,400.0,65.00,17.50,7.50,1.600,2.900,3.300,0.145,3.185,10.0,4.0,6.0,0.0,0.0,0.125,0.0,0.0,0.0,1.000,3.2,0.700,0.125,1.500,32.0,0.200,0.0,29.9,0.065,0.000,0.0,0.437,0.0,4.00,144.0,4.000,0.0,0.452,0.371,34.0,0.019,0.000,3.20,408.0,405.0,694.0,1284.0,701.0,312.0,895.0,575.0,234.0,937.0,564.0,1179.0,872.0,1436.0,3718.0,846.0,923.0,744.0,0.0
Choline Bitartrate,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,550.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Wheat bran, crude",216.0,64.51,15.55,4.25,0.630,0.637,2.212,0.167,2.039,42.8,0.0,42.8,0.0,9.0,1.303,0.0,0.0,0.0,1.490,1.9,0.523,0.577,13.578,79.0,2.181,0.0,74.4,0.073,0.000,0.0,0.998,0.0,10.57,611.0,5.000,0.0,1.013,1.182,77.6,0.002,0.605,7.27,371.0,430.0,486.0,928.0,600.0,234.0,595.0,500.0,282.0,726.0,436.0,1087.0,765.0,1130.0,2874.0,898.0,882.0,684.0,0.0
Lecithin granules,700.0,10.00,0.00,50.00,15.000,0.000,35.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,13000.0,0.200,0.000,0.0,0.000,0.0,3.60,0.0,0.000,0.0,3.000,1.600,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alpha-galactosidase enzyme (Beano),0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Canola oil,884.0,0.00,0.00,100.00,7.365,63.276,28.142,9.137,19.005,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,17.460,71.3,0.000,0.000,0.000,0.0,0.000,0.0,0.2,0.000,0.000,0.0,0.000,0.0,0.00,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pea Protein Powder,375.0,1.00,82.10,7.14,3.570,0.000,0.000,0.000,0.000,0.5,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.0,95.0,0.143,0.000,0.0,0.000,0.0,5.79,0.0,0.000,0.0,0.000,0.000,0.0,1.200,0.000,0.00,870.0,2510.0,4760.0,8410.0,7360.0,980.0,5520.0,4050.0,1030.0,5030.0,2810.0,6540.0,3210.0,9110.0,13530.0,3080.0,3380.0,4170.0,0.0
Multivitamin,0.0,0.00,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,3500.0,2.000,6.0,90.0,400.0,30.000,25.0,1.500,1.700,20.000,500.0,10.00